In [1]:
import ee
import geemap
ee.Initialize()
Map = geemap.Map()

In [2]:
roi = ee.Geometry.Polygon([[[85.5514759136554, 27.606319858773542], [85.54812851680481, 27.606814241656696],[85.54658356441223, 27.606053651681712], [85.54458800090515, 27.60747975355365],[85.5417341305133, 27.6071565054229], [85.53602638972961, 27.607574826351808],[85.53164902461731, 27.604608516183408], [85.52461090816223, 27.59989267830524],[85.52083435786926, 27.595633037415077], [85.51439705623352, 27.59160143910352],[85.51250878108704, 27.587113259052792], [85.51688614619934, 27.580799067650698],[85.53018990291321, 27.571098848697872], [85.53345146907532, 27.56934891397023],[85.53911629451477, 27.570680388584304], [85.54323616756164, 27.57018584275639],[85.54615441096985, 27.571441223967476], [85.55173340572082, 27.577299479711662],[85.55516663325989, 27.583461722809357], [85.55645409358704, 27.586923077874847],[85.55645409358704, 27.591221091986892], [85.55808487666809, 27.594301865688895],[85.55911484492981, 27.600272995334457], [85.55937233699524, 27.601680156867083],[85.56040230525696, 27.60426624445798],  [85.56070271266663, 27.60685227101685],[85.5598873211261, 27.60837343459905],  [85.5573124004718, 27.607726942657226],[85.55464092029297, 27.60593005531206], [85.55328908847623, 27.605834992139737]]])

In [7]:
def tof( roi, npixels_to_expand_other_classes=3, npixels_to_expand_tof=5, npixels_to_expand_forest=8, scale=10, niterations=1):
    lc = ee.ImageCollection('ESA/WorldCover/v200').first().clip(roi)
    lc_reclassified = lc.remap([10], to= [1], defaultValue = 0)
    tree_cover_mask = lc_reclassified.eq(1)
    tree_cover = lc_reclassified.updateMask(tree_cover_mask)
    other_classes_mask = lc_reclassified.eq(0)
    other_classes = lc_reclassified.updateMask(other_classes_mask)

    def expand(image_to_expand, npixels_to_expand, expand_value, background_values, roi, scale):
        if roi is not None:
            vectors = image_to_expand.select(0).int8().reduceToVectors(geometry = roi, scale = scale, maxPixels = 10e35)
        else:
            vectors = image_to_expand.select(0).int8().reduceToVectors(geometry = image_to_expand.geometry(), scale = scale, maxPixels = 10e35)
        
        
        def buffer(feature):
            feature = ee.Feature(feature)
            buffer = feature.buffer(npixels_to_expand*scale, 0.2)
            return buffer
        vectors_buffer = vectors.filter(ee.Filter.eq('label', expand_value)).map(lambda feature: buffer(feature))

        vectors_merged = ee.FeatureCollection([])
        for background_value in background_values:
            vectors_merged = vectors_merged.merge(vectors.filter(ee.Filter.eq('label', background_value)))
        
        final_merge = vectors_buffer.merge(vectors_merged)
        if roi is not None:
            expanded_image = final_merge.reduceToImage(['label'], ee.Reducer.first()).clip(roi)
        else:
            expanded_image = final_merge.reduceToImage(['label'], ee.Reducer.first()).clip(image_to_expand.geometry())

        return expanded_image
    
    def id_tof():
        expanded_other_classes = expand(
            image_to_expand = other_classes_mask, 
            expand_value = 1, 
            background_values = [0], 
            npixels_to_expand = npixels_to_expand_other_classes, 
            scale = 10, 
            roi = roi
            )

        add_expanded_other_classes_lc_reclassified = lc_reclassified.add(expanded_other_classes.remap([0,1], [1, 0]))

        expanded_tof = expand(
            image_to_expand = add_expanded_other_classes_lc_reclassified, 
            expand_value = 1, 
            background_values = [0, 2], 
            npixels_to_expand = npixels_to_expand_tof, 
            scale = 10, 
            roi = roi
            )

        expanded_forest = expand(
            image_to_expand = expanded_tof, 
            expand_value = 2, 
            background_values = [0, 1], 
            npixels_to_expand = npixels_to_expand_forest, 
            scale = 10, 
            roi = roi
            )
        
        for i in range(niterations - 1):
            expanded_other_classes = expand(
                image_to_expand = expanded_other_classes, 
                expand_value = 1, 
                background_values = [0], 
                npixels_to_expand = npixels_to_expand_other_classes, 
                scale = 10, 
                roi = roi
                )

            add_expanded_other_classes_lc_reclassified = lc_reclassified.add(expanded_other_classes.remap([0,1], [1, 0]))

            expanded_tof = expand(
                image_to_expand = add_expanded_other_classes_lc_reclassified, 
                expand_value = 1, 
                background_values = [0, 2], 
                npixels_to_expand = npixels_to_expand_tof, 
                scale = 10, 
                roi = roi
                )

            expanded_forest = expand(
                image_to_expand = expanded_tof, 
                expand_value = 2, 
                background_values = [0, 1], 
                npixels_to_expand = npixels_to_expand_forest, 
                scale = 10, 
                roi = roi
                )

        forest_vectors = tree_cover.select(0).int8().reduceToVectors(geometry = roi, scale = 10, maxPixels = 10e35)
        tof_and_forest = expanded_forest.clip(forest_vectors)
        
        return tof_and_forest
    return id_tof()

<h4>Adjust the Parameters to Properly Identify Forest and Trees Outside Forest</h4>

In [4]:
#Using all the default parameters
tof_and_forest = tof(roi = roi)

In [5]:
Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.addLayer(tof_and_forest, {'min':1, 'max':2, 'palette':['blue', 'green']}, 'tof')

In [6]:
Map.center_object(roi)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…